In [1]:
import pya
import numpy as np




In [2]:

def create_poly_dicing_corner(length, width):
    poly = pya.DPolygon([ 
    pya.DPoint(0, 0), pya.DPoint(0, length),  pya.DPoint(width, length),
    pya.DPoint(width, width),  pya.DPoint(length, width), pya.DPoint(length, width),
    pya.DPoint(length, 0)
    ])
    return poly

def create_corners_cell(layout, field_width, field_height, corner_length, corner_width, layer="layer127"):
    cell_corner_single = layout.create_cell('corner_instance')

    cell_corner = layout.create_cell('corners')
    layer_corners = layout.layer(layer)
    corner_polygon = create_poly_dicing_corner(corner_length, corner_width)

    cell_corner_single.shapes(layer_corners).insert(corner_polygon)


    instance = pya.DCellInstArray(cell_corner_single.cell_index(),
                                pya.DTrans(pya.DTrans.R0, pya.DPoint(-field_width/2, -field_height/2))
                                )
    cell_corner.insert(instance)

    instance = pya.DCellInstArray(cell_corner_single.cell_index(),
                                pya.DTrans(pya.DTrans.R270, pya.DPoint(-field_width/2, field_height/2))
                                )
    cell_corner.insert(instance)


    instance = pya.DCellInstArray(cell_corner_single.cell_index(),
                                pya.DTrans(pya.DTrans.R180, pya.DPoint(field_width/2, field_height/2))
                                )
    cell_corner.insert(instance)

    instance = pya.DCellInstArray(cell_corner_single.cell_index(),
                                pya.DTrans(pya.DTrans.R90, pya.DPoint(field_width/2, -field_height/2))
                                )
    cell_corner.insert(instance)


    cell_corner.flatten(-1, True)

    return cell_corner



def create_label_cell(layout, text, position=(0,0), rotate=True, mag=1000, layer="layer127"):
    

    cell_label = layout.create_cell('label')
    layer_label = layout.layer(layer)


    gen = pya.TextGenerator.default_generator()
    
    region = gen.text(text, layout.dbu, mag)
    if rotate:
        cell_label.shapes(layer_label).insert(region, pya.DTrans(pya.DTrans.R90,pya.DVector(position[0], position[1])))
    else:
        cell_label.shapes(layer_label).insert(region, pya.DTrans(pya.DVector(position[0], position[1])))
    return cell_label

In [47]:

#initialize
layout = pya.Layout()
top = layout.create_cell('top')



# single_calibration = layout.create_cell('rectangle')
# layer_n = layout.layer("layer_n")


layers = np.arange(0,128)




# Create matrix of squares for measurement
cell_gs_matrix = layout.create_cell('gs_matrix')

step = 3000.0
square_side = 2000

rows = range(8)
cols = range(16)

x0,y0 = -24000+500, 9000+500+4500


start_grayvalue = 48

CC,RR = np.meshgrid(cols, rows, indexing='xy')
# top_cell = layout.top_cell()

for i, (layer, col, row) in enumerate(zip(layers, CC.flatten(), RR.flatten())):

    if i<start_grayvalue:
        continue
    # print(i, layer, row, col)
    layer_name = "layer%03d"%(layer)

    layer_i = layout.layer(layer_name)

    x,y = step*col, -step*row
    x += x0
    y += y0
    # print(x,y, x+square_side, y+square_side)


    # new_instance = pya.CellInstArray(cell_single_level.cell_index(), trans, pya.Vector(50, 0 ), pya.Vector(0, 50), 16, 8 )
    # cell_single_level.shapes(layer_i).insert(pya.Box(square_side, square_side).move(pya.Vector(dx, dy)))
    # cell_single_level.shapes(layer_i).insert(pya.DBox(square_side, square_side))
    cell_gs_matrix.shapes(layer_i).insert(pya.DBox(x,y, x+square_side, y+square_side))


    
    position=(x, y-250)
    rotate=False
    mag=200

    label = "GS%d"%(i)
    trans = pya.Trans(pya.Point(0,0))
    layer = "layer127"

    cell_label = create_label_cell(layout, label, position, rotate, mag, layer)
    top.insert(pya.DCellInstArray(cell_label.cell_index(), trans))
trans = pya.Trans(pya.Point(0,0))

new_instance = pya.DCellInstArray(cell_gs_matrix.cell_index(), trans, pya.Vector(50, 0 ), pya.Vector(0, 50), 1,1)
# reference cell index, transformation, x offset, y offset, number of x instances, and number of y instances

# calibration_instance = pya.Instance()
top.insert(new_instance)

total_width = 780



corner_length = 500#*micro
corner_width = 100#*micro

field_width = 48000 #*micro
field_height = 25000 #*micro
field_height = 15000 #*micro

layer = "layer127"
# gscalibration.add_corners(field_width, field_height, corner_length, corner_width, layer)
label = "text"
# gscalibration.add_label(label, position=(-4000, -4000), mag=400, layer=layer)


trans = pya.Trans(pya.Point(0,0))

top_cell = layout.top_cell()

cell_corner = create_corners_cell(layout, field_width, field_height, corner_length, corner_width, layer)
top_cell.insert(pya.DCellInstArray(cell_corner.cell_index(), trans))


# for i in range (16):
#     x0,y0 = -22500, 7250
#     position=(x0+step*i, y0)
#     rotate=False
#     mag=200

#     label = "%d"%(i)
#     cell_label = create_label_cell(layout, label, position, rotate, mag, layer)
#     top_cell.insert(pya.DCellInstArray(cell_label.cell_index(), trans))


layout.write("test128.dxf")



# layout.write("gs_calibration_v20240507.dxf")

In [4]:
import numpy as np

# Set the seed for reproducibility
np.random.seed(42)

# Create the array
arr = np.random.randint(0, 128, size=(8,16))

# Print the array
print(arr)


[[102  51  92  14 106  71  60  20 102 121  82  86  74  74  87 116]
 [ 99 103  23   2  21  52   1  87 107  29  37   1  63  59  20  32]
 [ 75  57  21 124 107  88  48  90  58 126  41 127  91  59  79  14]
 [ 61  61  46  61  50 107  54 115  63 120   2 100  50   6  20  72]
 [ 38  17   3  88  59  13 113 121   8  89  52   1  83  91 110  59]
 [ 70  43 124   7  46  34  77  80  35  49 103   3   1 125   5  53]
 [105   3  53  92  62  17  89  43  33  73  61  99  13  94  47  14]
 [ 71  77  86 123 120  61  39  84  79 108  81 110  52  23  25  88]]


In [ ]:
1.4 um com 10 niveis




for i in [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,64,127,]:


In [ ]:
target = 1.4/10


In [ ]:

#initialize
layout = pya.Layout()
top = layout.create_cell('top')
c_circle = layout.create_cell('circle')
l_circle = layout.layer("layer000")

# make a circle pcell
radius = 200 #radius in nm
nr_points = 32 #number of points
#create an array of angles
angles = np.linspace(0,2*np.pi,nr_points+1)[0:-1]
points = [] #array of point
for angle in angles:
    points.append(pya.Point(radius*np.cos(angle),radius*np.sin(angle)))
circle = pya.SimplePolygon(points)



#for hexagonal grid
a = 400 #hexagon radius towards flat
#number of instances
i=10
c_circle.shapes(l_circle).insert(circle)
c_circle.shapes(l_circle).insert(circle.moved(a*np.sqrt(3),a))



trans = pya.Trans(pya.Point(0,0))
new_instance = pya.CellInstArray(c_circle.cell_index(),trans,pya.Vector(2*a*np.sqrt(3), 0 ), pya.Vector(0, 2*a), i/2, i/2)
top.insert(new_instance)
layout.write("test.dxf")